In [33]:
import requests
from pyquery import PyQuery as pq
import pandas as pd
import re


def get_buildings():
    response = requests.get(
        'https://cookieclicker.fandom.com/wiki/Buildings')
    html = pq(response.content)
    buildings = pd.read_html(html('.wikitable').outer_html(), index_col=0)[0][
        ['Base Cost', 'Base CpS']]
    return buildings


def get_upgrades(buildings):
    response = requests.get(
        'https://cookieclicker.fandom.com/wiki/Upgrades')
    html = pq(response.content)
    captions = html('caption')
    upgrades = []
    for name in buildings.index:
        regex = re.compile(
            '\d+ (?={})'.format(name[:-1]), re.IGNORECASE)
        table = captions(':contains("%s")' % name).parent()
        requirement = table('td:nth-child(3)')
        requirement.map(lambda i, e: pq(e).text(
            regex.search(pq(e).text()).group(0)))

        price = table('td:nth-child(4) span:nth-child(2)')
        price.map(lambda i, e: pq(e).text(pq(e).attr('title')))
        price.children().remove()
        upgrades.append(pd.read_html(
            table.outer_html())[0][['Name', 'Unlock condition', 'Base price']])
    upgrades = pd.concat(upgrades, keys=buildings.index, axis=1)
    return upgrades

buildings = get_buildings()
upgrades = get_upgrades(buildings)
# print(upgrades)
buildings.to_excel('buildings.xlsx')
upgrades.to_excel('upgrades.xlsx')
